In [1]:
!pip install pandas pyarrow

In [2]:
import os
import numpy as np
print(os.getcwd())

import pandas as pd

file_path = 'nc_reduced_data.feather'
data = pd.read_feather(file_path)
print(data.head())

/Users/fasihz/Duke/COMPSCI 526: Data Science
             datetime       file_source county_name_cat  subject_age  \
0 2000-01-11 23:30:00  nc_winston-salem             NaN         49.0   
1 2000-01-12 00:59:00  nc_winston-salem             NaN         21.0   
2 2000-01-12 21:05:00  nc_winston-salem             NaN         22.0   
3 2000-01-12 21:05:00  nc_winston-salem             NaN         21.0   
4 2000-01-12 22:37:00  nc_winston-salem             NaN         19.0   

  subject_sex_cat subject_race_cat raw_ethnicity_cat  \
0          female            black                 N   
1            male            black                 N   
2            male            white                 N   
3          female            black                 N   
4          female            black                 N   

                                department_name_cat outcome_cat  \
0  Winston-Salem State University Police Department     warning   
1  Winston-Salem State University Police Department

In [3]:
print(data.columns)
for column in data.columns:
    unique_values = data[column].unique()
    print(f"columns: {column}")
    print(f"number of unique values: {len(unique_values)}")
    print(f"unique values: {unique_values}")

Index(['datetime', 'file_source', 'county_name_cat', 'subject_age',
       'subject_sex_cat', 'subject_race_cat', 'raw_ethnicity_cat',
       'department_name_cat', 'outcome_cat', 'raw_action_description_cat',
       'contraband_found_cat', 'contraband_drugs_cat',
       'contraband_weapons_cat', 'reason_for_frisk_cat',
       'reason_for_search_cat', 'reason_for_stop_cat'],
      dtype='object')
columns: datetime
number of unique values: 5680426
unique values: <DatetimeArray>
['2000-01-11 23:30:00', '2000-01-12 00:59:00', '2000-01-12 21:05:00',
 '2000-01-12 22:37:00', '2000-01-12 23:05:00', '2000-01-13 23:50:00',
 '2000-01-12 23:30:00', '2000-01-15 18:45:00', '2000-01-30 01:20:00',
 '2000-01-28 08:57:00',
 ...
 '2015-12-08 13:49:00', '2010-12-08 19:59:00', '2015-12-31 04:40:00',
 '2015-10-26 16:27:00', '2015-10-27 15:52:00', '2015-12-30 19:15:00',
 '2015-11-12 13:48:00', '2015-09-28 08:51:00', '2015-12-26 17:01:00',
 '2015-10-30 19:27:00']
Length: 5680426, dtype: datetime64[ns]
column

In [4]:
# This code first finds rows where 'outcome_cat' is NaN, and check if 'raw_action_description_cat' is 'No Action Taken' for all these rows

# Find rows where 'outcome_cat' is NaN
nan_rows = data[data['outcome_cat'].isna()]

# Check if 'raw_action_description_cat' is 'No Action Taken' for all these rows
mismatch_indices = nan_rows[nan_rows['raw_action_description_cat'] != 'No Action Taken'].index

if len(mismatch_indices) > 0:
    print(f"Mismatch found at indices: {mismatch_indices.tolist()}")
else:
    print("All instances match: 'No Action Taken' where 'outcome_cat' is NaN")

All instances match: 'No Action Taken' where 'outcome_cat' is NaN


In [5]:
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# I defined all possible categories upfront for consistency
all_race_categories = ['black', 'white', 'hispanic', 'asian/pacific islander', 'unknown']
all_sex_categories = ['female', 'male', 'unknown']
all_outcome_categories = ['warning', 'citation', 'arrest', 'no_action']

# I converted categorical columns to pd.Categorical with predefined categories
data['subject_race_cat'] = pd.Categorical(data['subject_race_cat'], categories=all_race_categories)
data['subject_sex_cat'] = pd.Categorical(data['subject_sex_cat'], categories=all_sex_categories)
data['outcome_cat'] = pd.Categorical(data['outcome_cat'], categories=all_outcome_categories)

# I replaced NaN, 'unknown', and 'other' in 'subject_race_cat' with 'unknown'
data['subject_race_cat'] = data['subject_race_cat'].replace(['unknown', 'other'], 'unknown')
data['subject_race_cat'] = data['subject_race_cat'].fillna('unknown')

# Next, I replaced NaN in 'subject_sex_cat' with 'unknown' (Just for convenience)
data['subject_sex_cat'] = data['subject_sex_cat'].fillna('unknown')

import statsmodels.api as sm

#Replace NaN in 'outcome_cat' with 'no_action'
data['outcome_cat'] = data['outcome_cat'].fillna('no_action').copy()

# I also removed rows with NaN values in 'subject_age'**
data = data.dropna(subset=['subject_age'])

# Define numeric and categorical features
numeric_features = ['subject_age']
categorical_features = ['subject_race_cat', 'subject_sex_cat', 'file_source']

# Standardize numeric features
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(data[numeric_features])

# Convert the scaled numeric data to a DataFrame
X_numeric_df = pd.DataFrame(X_numeric_scaled, columns=numeric_features)

# One-hot encode categorical features
onehot = OneHotEncoder(drop='first', sparse_output=True)
X_categorical_encoded = onehot.fit_transform(data[categorical_features])

# Convert the one-hot encoded data to a DataFrame
categorical_col_names = onehot.get_feature_names_out(categorical_features)
# Convert to a sparse DataFrame directly
X_categorical_df = pd.DataFrame.sparse.from_spmatrix(X_categorical_encoded, columns=onehot.get_feature_names_out(categorical_features))

# Combine numeric and categorical features into one DataFrame using pandas.concat()
X_combined = pd.concat([X_numeric_df.reset_index(drop=True), X_categorical_df.reset_index(drop=True)], axis=1)

# Add an intercept term for logistic regression
X_combined['constant'] = 1
X_combined

,subject_age,subject_race_cat_black,subject_race_cat_hispanic,subject_race_cat_unknown,subject_race_cat_white,subject_sex_cat_male,file_source_nc_durham,file_source_nc_fayetteville,file_source_nc_greensboro,file_source_nc_raleigh,file_source_nc_statewide,file_source_nc_winston-salem,constant
0,1.050343,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,-1.036153,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,-0.961636,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
3,-1.036153,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,-1.185189,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24604614,-1.259706,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1
24604615,-0.290976,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1
24604616,-1.036153,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
24604617,-1.036153,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1


In [6]:
import statsmodels.api as sm

# Create the dependent variable (outcome) as a categorical variable
data['outcome_cat'] = data['outcome_cat'].astype('category')

# Prepare Y variable
Y = data['outcome_cat'].cat.codes  # Convert to codes (0, 1, 2, 3)

Y

0           0
1           1
2           0
3           1
4           0
           ..
24607106    1
24607107    2
24607108    1
24607109    0
24607110    0
Length: 24604619, dtype: int8

In [7]:
print(data['subject_age'].describe())  # Check the distribution of subject_age

print(data['outcome_cat'].value_counts())  # Check the distribution of outcome categories

count    2.460462e+07
mean     3.490479e+01
std      1.341963e+01
min      1.000000e+01
25%      2.400000e+01
50%      3.200000e+01
75%      4.400000e+01
max      1.100000e+02
Name: subject_age, dtype: float64
outcome_cat
citation     15881476
warning       7436974
no_action      744819
arrest         541350
Name: count, dtype: int64


In [8]:
print(X_combined.nunique())

subject_age                     100
subject_race_cat_black            2
subject_race_cat_hispanic         2
subject_race_cat_unknown          2
subject_race_cat_white            2
subject_sex_cat_male              2
file_source_nc_durham             2
file_source_nc_fayetteville       2
file_source_nc_greensboro         2
file_source_nc_raleigh            2
file_source_nc_statewide          2
file_source_nc_winston-salem      2
constant                          1
dtype: int64


In [9]:
correlation_matrix = X_combined.corr()
print(correlation_matrix)

'file_source_nc_winston-salem', 

                              subject_age  subject_race_cat_black  \
subject_age                      1.000000               -0.024369   
subject_race_cat_black          -0.024369                1.000000   
subject_race_cat_hispanic       -0.089280               -0.210798   
subject_race_cat_unknown        -0.012386               -0.098745   
subject_race_cat_white           0.076058               -0.792258   
subject_sex_cat_male             0.018788               -0.042926   
file_source_nc_durham           -0.000092                0.053015   
file_source_nc_fayetteville     -0.016001                0.067393   
file_source_nc_greensboro       -0.011703                0.053980   
file_source_nc_raleigh          -0.017152                0.044028   
file_source_nc_statewide         0.025971               -0.152229   
file_source_nc_winston-salem     0.001658                0.030778   
constant                              NaN                     NaN   

                              sub

('file_source_nc_winston-salem',)

In [10]:
# Running regression on all the variables gives us a singular matrix (perfect multicollinearity).
# This is a big issue, so clearly we need to do feature reduction. 
# After a lot of trial and error and VIF tests, it became clear that the file_source variables were problematic.
# Along with that, having a variable for each race creates a dummy variable trap which also gives us spurrious regression results. 
# Therefore, we have dropped the white race column and now have that as the reference category. 
# This allows us to compare whites with other races when it comes to stops and outcomes. Very good reference point.
X_combined_3 = X_combined.drop(columns=['file_source_nc_winston-salem','file_source_nc_statewide', 'subject_race_cat_white', 'file_source_nc_fayetteville', 'file_source_nc_greensboro', 'file_source_nc_durham', 'file_source_nc_raleigh'])
X_combined_3

,subject_age,subject_race_cat_black,subject_race_cat_hispanic,subject_race_cat_unknown,subject_sex_cat_male,constant
0,1.050343,1.0,0.0,0.0,0.0,1
1,-1.036153,1.0,0.0,0.0,1.0,1
2,-0.961636,0.0,0.0,0.0,1.0,1
3,-1.036153,1.0,0.0,0.0,0.0,1
4,-1.185189,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...
24604614,-1.259706,0.0,0.0,0.0,1.0,1
24604615,-0.290976,0.0,0.0,0.0,1.0,1
24604616,-1.036153,0.0,0.0,0.0,0.0,1
24604617,-1.036153,0.0,0.0,0.0,1.0,1


In [11]:
# Sample a smaller subset (e.g., first 10000 rows)
X_sample = X_combined_3[:10000]
Y_sample = Y[:10000]

logit_model = sm.MNLogit(Y_sample, X_sample)
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.812801
         Iterations 8
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9982
Method:                           MLE   Df Model:                           15
Date:                Wed, 23 Oct 2024   Pseudo R-squ.:                 0.01759
Time:                        00:26:04   Log-Likelihood:                -8128.0
converged:                       True   LL-Null:                       -8273.5
Covariance Type:            nonrobust   LLR p-value:                 4.156e-53
                      y=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
subject_age                  -0.1993      0.022     -9.193      0.000      -0.

In [12]:
#variation inflation factors seem fine now! None are above 10. 

from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.Series([variance_inflation_factor(X_sample, i) for i in range(X_sample.shape[1])], 
                index=X_sample.columns)
print(vif)

subject_age                  1.033237
subject_race_cat_black       1.104013
subject_race_cat_hispanic    1.156812
subject_race_cat_unknown     1.009749
subject_sex_cat_male         1.024916
constant                     3.632098
dtype: float64


In [13]:
# Sample a bigger subset (e.g., 100000 rows)
X_sample_big = X_combined_3[:100000]
Y_sample_big = Y[:100000]

multinomial_logit_model_big = sm.MNLogit(Y_sample_big, X_sample_big)
result_2 = multinomial_logit_model_big.fit()
print(result_2.summary())

Optimization terminated successfully.
         Current function value: 0.656535
         Iterations 8
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:               100000
Model:                        MNLogit   Df Residuals:                    99982
Method:                           MLE   Df Model:                           15
Date:                Wed, 23 Oct 2024   Pseudo R-squ.:                 0.01191
Time:                        00:27:26   Log-Likelihood:                -65653.
converged:                       True   LL-Null:                       -66445.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                      y=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
subject_age                  -0.1847      0.008    -24.612      0.000      -0.

In [21]:
# Sample biggest subset (e.g., 500000 rows)
X_sample_biggest = X_combined_3[:390000]
Y_sample_biggest = Y[:390000]

multinomial_logit_model_biggest = sm.MNLogit(Y_sample_biggest, X_sample_biggest)
result_3 = multinomial_logit_model_biggest.fit()
print(result_3.summary())

Optimization terminated successfully.
         Current function value: 0.725190
         Iterations 8
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:               390000
Model:                        MNLogit   Df Residuals:                   389982
Method:                           MLE   Df Model:                           15
Date:                Wed, 23 Oct 2024   Pseudo R-squ.:                 0.01233
Time:                        00:41:13   Log-Likelihood:            -2.8282e+05
converged:                       True   LL-Null:                   -2.8636e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                      y=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
subject_age                  -0.1879      0.004    -51.845      0.000      -0.

In [ ]:
# Run the logistic regression on the shuffled data
multi_logit_model = sm.MNLogit(Y_shuffled, X_shuffled)
result_shuffled = multi_logit_model.fit()

# Print the summary
print(result_shuffled.summary())

In [ ]:
# Sample a smaller subset (e.g., first 1000 rows)
X_sample_big = X_combined_3[:100000]
Y_sample_big = Y[:100000]

multinomial_logit_model_big = sm.MNLogit(Y_sample_big, X_sample_big)
result_2 = multinomial_logit_model_big.fit()
print(result_2.summary())

In [68]:
from sklearn.linear_model import LogisticRegression

# Fit the model using scikit-learn with regularization
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', C=1.0, max_iter=1000)
logistic_model.fit(X_combined_3, Y)
print(logistic_model.coef_)

#This regression does not give us the best results. 
#But it does run. Our dataset is huge so we tried a few different methods. 

/Users/fasihz/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[[ 0.06916639 -0.06187874 -0.34908951 -0.07499743 -0.41691289  0.70513335]
 [-0.06036974 -0.26783809 -0.06645657  0.19209921 -0.34443564  1.08572084]
 [-0.27919298  0.29392939  0.55477186 -0.07944947  0.53763968 -1.07882913]
 [ 0.27039632  0.03578744 -0.13922579 -0.03765231  0.22370885 -0.71202506]]


In [ ]:
#Now we will try running a the multinomial logistic regression on all the data. No sampling.
# This will take hours, but let's see.

# Reset the indices to align them
Y_aligned = Y.reset_index(drop=True)
X_aligned = X_combined_3.reset_index(drop=True)

# Run the logistic regression
multi_logit_model = sm.MNLogit(Y_aligned, X_aligned)
result_full = multi_logit_model.fit()

# Print the summary
print(result_full.summary())

Optimization terminated successfully.
         Current function value: 0.824234
         Iterations 8
